# Parameter optimization

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'buy'
pattern = ['Pattern', 'Trend']
indicator_list = pattern
indicator_list_higher = pattern

work_timeframe = '15m'
higher_timeframe = '1h'
opt_limit = 2000
load = False

print(f'Timeframe is {higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Indicator_list'] = pattern
configs['Higher_TF_indicator_list'] = indicator_list_higher
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {
                'Pattern': {
                            'use_vol': [-1], 
                            'window_low_bound': [1, 2, 3], 
                            'window_high_bound': [4, 5, 6, 7],
                            'first_candle': [0.6],
                            'second_candle': [0.55],
                            'third_candle': [0.5]
                            },
                'Trend': {
                            'timeperiod': [28],
                            'low_bound': [0]
                            }
             }

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

# stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
# if not stat_list:
#     stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
# else:
#     stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')


Timeframe is 1h, trade type is buy
Number of combinations is 12


  0%|                                                                                                                                                                                                   | 0/12 [00:00<?, ?it/s]

# Check local statistics

In [6]:
# candles
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_rank', ascending=False)
stat.head(20)

,pattern,Pattern_use_vol,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Pattern_third_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
0,Pattern_Trend,-1,1,5,0.50,0.50,0.5,28,0,1677,2.007387,2.360000,1689.388837,3957.720000,1.4016,1.3949,1.4406,1.5140,1.5936,1.6565,1.7743,1.8254,1.8523,1.9076,1.9315,1.9521,2.1624,2.1931,2.2171,2.2445,2.2932,2.3415,2.3732,2.3983,2.4104,2.4328,2.4254,2.4410,1.43,1.37,1.53,1.63,1.75,1.84,1.94,2.00,2.08,2.16,2.19,2.26,2.37,2.46,2.57,2.69,2.76,2.86,2.96,3.03,3.09,3.17,3.21,3.29
10,Pattern_Trend,-1,1,5,0.55,0.50,0.5,28,0,1580,2.017004,2.387500,1606.866583,3772.250000,1.4085,1.4081,1.4686,1.5486,1.6055,1.6836,1.7856,1.8309,1.8546,1.9106,1.9164,1.9410,2.1668,2.1957,2.2183,2.2515,2.3026,2.3519,2.3824,2.4074,2.4245,2.4481,2.4387,2.4582,1.46,1.42,1.59,1.69,1.82,1.90,2.00,2.04,2.13,2.20,2.23,2.29,2.39,2.47,2.58,2.70,2.77,2.87,2.97,3.03,3.08,3.17,3.21,3.29
1,Pattern_Trend,-1,1,5,0.50,0.55,0.5,28,0,1577,2.033463,2.385417,1629.770363,3761.802083,1.4240,1.4071,1.4487,1.5248,1.6132,1.6701,1.7965,1.8501,1.8792,1.9242,1.9502,1.9658,2.1860,2.2166,2.2488,2.2749,2.3247,2.3772,2.4143,2.4425,2.4527,2.4741,2.4647,2.4727,1.48,1.39,1.56,1.66,1.78,1.87,1.97,2.04,2.12,2.19,2.23,2.30,2.40,2.48,2.59,2.71,2.78,2.88,2.98,3.04,3.11,3.18,3.22,3.29
20,Pattern_Trend,-1,1,5,0.60,0.50,0.5,28,0,1479,2.035029,2.400417,1530.808137,3550.216250,1.4104,1.4123,1.4786,1.5672,1.6268,1.7048,1.8149,1.8386,1.8602,1.9226,1.9328,1.9552,2.1939,2.2230,2.2444,2.2855,2.3348,2.3903,2.4140,2.4204,2.4356,2.4574,2.4466,2.4704,1.42,1.38,1.56,1.68,1.81,1.90,2.01,2.06,2.14,2.22,2.25,2.31,2.42,2.51,2.61,2.73,2.81,2.90,2.99,3.05,3.11,3.19,3.24,3.31
11,Pattern_Trend,-1,1,5,0.55,0.55,0.5,28,0,1462,1.988558,2.408333,1445.272283,3520.983333,1.4137,1.4088,1.4503,1.5121,1.5781,1.6352,1.7301,1.7784,1.8073,1.8598,1.8689,1.8893,2.1283,2.1572,2.1884,2.2217,2.2763,2.3306,2.3680,2.3980,2.4145,2.4399,2.4288,2.4417,1.51,1.43,1.60,1.70,1.83,1.91,2.01,2.06,2.15,2.22,2.27,2.32,2.42,2.50,2.61,2.72,2.80,2.89,2.99,3.05,3.11,3.18,3.23,3.29
2,Pattern_Trend,-1,1,5,0.50,0.60,0.5,28,0,1483,2.028117,2.350417,1524.697017,3485.667917,1.3977,1.3812,1.4394,1.5202,1.6146,1.6740,1.7970,1.8532,1.8732,1.9109,1.9398,1.9457,2.1741,2.2028,2.2403,2.2686,2.3185,2.3698,2.4158,2.4463,2.4610,2.4807,2.4684,2.4816,1.40,1.32,1.50,1.59,1.72,1.81,1.92,2.01,2.09,2.14,2.20,2.27,2.37,2.45,2.57,2.68,2.76,2.86,2.96,3.02,3.09,3.17,3.22,3.29
21,Pattern_Trend,-1,1,5,0.60,0.55,0.5,28,0,1372,2.007992,2.427500,1382.964567,3330.530000,1.4175,1.4164,1.4621,1.5306,1.5983,1.6556,1.7617,1.7911,1.8181,1.8755,1.8886,1.9058,2.1590,2.1871,2.2165,2.2562,2.3086,2.3704,2.4009,2.4108,2.4248,2.4483,2.4351,2.4528,1.47,1.39,1.58,1.69,1.83,1.91,2.03,2.08,2.17,2.25,2.30,2.35,2.46,2.54,2.64,2.76,2.84,2.93,3.03,3.08,3.14,3.21,3.26,3.32
30,Pattern_Trend,-1,1,5,0.65,0.50,0.5,28,0,1376,1.943438,2.395833,1298.170000,3296.666667,1.3790,1.3775,1.3566,1.4457,1.5098,1.5781,1.6802,1.7182,1.7482,1.7995,1.8127,1.8458,2.1050,2.1374,2.1547,2.2030,2.2532,2.3129,2.3378,2.3504,2.3681,2.3902,2.3805,2.3980,1.45,1.38,1.56,1.69,1.82,1.92,2.02,2.06,2.15,2.23,2.26,2.32,2.42,2.50,2.60,2.71,2.79,2.88,2.97,3.03,3.09,3.16,3.21,3.28
3,Pattern_Trend,-1,1,5,0.50,0.65,0.5,28,0,1396,2.003696,2.350000,1401.15

In [4]:
# LR timeperiod
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg', ascending=False)
stat.head(20)

,pattern,Pattern_use_vol,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Pattern_third_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
13,Pattern_Trend,-1,1,5,0.55,0.5,0.5,28,0,1707,2.022646,2.424167,1745.656437,4138.052500,1.4020,1.4110,1.4634,1.5375,1.6011,1.6794,1.7809,1.8337,1.8613,1.9181,1.9187,1.9451,2.1618,2.1930,2.2270,2.2610,2.3112,2.3620,2.3874,2.4195,2.4413,2.4635,2.4673,2.4963,1.48,1.42,1.60,1.71,1.85,1.94,2.04,2.08,2.18,2.24,2.28,2.33,2.43,2.51,2.62,2.73,2.81,2.91,3.01,3.07,3.13,3.21,3.26,3.34
18,Pattern_Trend,-1,1,5,0.55,0.5,0.5,38,0,1782,2.018304,2.423750,1814.618025,4319.122500,1.3825,1.4166,1.4696,1.5430,1.6011,1.6790,1.7763,1.8335,1.8639,1.9182,1.9224,1.9502,2.1589,2.1918,2.2195,2.2504,2.3031,2.3508,2.3753,2.4090,2.4312,2.4535,2.4558,2.4837,1.48,1.43,1.62,1.73,1.87,1.96,2.05,2.10,2.20,2.25,2.28,2.34,2.44,2.51,2.61,2.71,2.79,2.89,2.99,3.06,3.11,3.19,3.24,3.32
37,Pattern_Trend,-1,1,5,0.55,0.5,0.5,76,0,1994,1.987296,2.420833,1968.667892,4827.141667,1.3470,1.3870,1.4462,1.5191,1.5792,1.6723,1.7602,1.8217,1.8468,1.8968,1.8991,1.9395,2.1272,2.1569,2.1789,2.2087,2.2649,2.3083,2.3386,2.3658,2.3836,2.4044,2.4100,2.4329,1.51,1.44,1.62,1.74,1.88,1.97,2.05,2.11,2.20,2.26,2.29,2.35,2.45,2.53,2.63,2.71,2.78,2.86,2.96,3.03,3.08,3.15,3.21,3.29
12,Pattern_Trend,-1,1,5,0.55,0.5,0.5,26,0,1699,2.015608,2.420000,1725.518558,4111.580000,1.3954,1.4057,1.4573,1.5339,1.5974,1.6756,1.7797,1.8331,1.8612,1.9166,1.9163,1.9404,2.1602,2.1896,2.2236,2.2515,2.3014,2.3489,2.3732,2.4050,2.4262,2.4485,2.4524,2.4815,1.49,1.42,1.61,1.72,1.85,1.94,2.04,2.08,2.18,2.24,2.28,2.33,2.43,2.51,2.62,2.73,2.81,2.90,3.00,3.06,3.11,3.19,3.23,3.31
35,Pattern_Trend,-1,1,5,0.55,0.5,0.5,72,0,1970,1.982646,2.415833,1935.812292,4759.191667,1.3485,1.3810,1.4407,1.5143,1.5734,1.6648,1.7551,1.8181,1.8434,1.8938,1.8970,1.9363,2.1249,2.1521,2.1738,2.2037,2.2590,2.3017,2.3316,2.3582,2.3783,2.3994,2.4058,2.4286,1.52,1.45,1.62,1.73,1.87,1.97,2.05,2.10,2.19,2.25,2.28,2.35,2.45,2.52,2.62,2.70,2.77,2.85,2.95,3.03,3.08,3.15,3.20,3.28
22,Pattern_Trend,-1,1,5,0.55,0.5,0.5,46,0,1843,1.999233,2.415833,1841.587033,4452.380833,1.3766,1.3970,1.4589,1.5374,1.5885,1.6731,1.7664,1.8254,1.8485,1.8953,1.9016,1.9360,2.1368,2.1677,2.1952,2.2291,2.2833,2.3283,2.3541,2.3815,2.4026,2.4229,2.4240,2.4514,1.49,1.42,1.61,1.73,1.87,1.95,2.03,2.08,2.17,2.23,2.26,2.32,2.43,2.51,2.62,2.72,2.80,2.89,2.99,3.06,3.10,3.18,3.22,3.30
39,Pattern_Trend,-1,1,5,0.55,0.5,0.5,80,0,2015,1.968442,2.415000,1951.409958,4866.225000,1.3434,1.3814,1.4350,1.5047,1.5656,1.6571,1.7441,1.8049,1.8307,1.8814,1.8814,1.9216,2.1066,2.1367,2.1586,2.1868,2.2429,2.2868,2.3175,2.3382,2.3548,2.3754,2.3820,2.4050,1.51,1.44,1.62,1.73,1.87,1.96,2.05,2.10,2.20,2.25,2.28,2.35,2.45,2.52,2.62,2.71,2.77,2.86,2.95,3.02,3.08,3.14,3.20,3.28
15,Pattern_Trend,-1,1,5,0.55,0.5,0.5,32,0,1740,2.017583,2.415000,1770.595000,4202.100000,1.3856,1.4092,1.4596,1.5336,1.5965,1.6754,1.7765,1.8283,1.8555,1.9119,1.9146,1.9439,2.1587,2.1896,2.2222,2.2564,2.3065,2.3575,2.3831,2.4168,2.4363,2.4580,2.4590,2.4873,1.48,1.43,1.61,1.72,1.86,1.94,2.04,2.08,2.18,2.23,2.26,2.32,2.42,2.50,2.60,2.71,2.79,2.89,2.99,3.05,3.11,3.19,3.24,3.32
33,Pattern_Trend,-1,1,5,0.55,0.5,0.5,68,0,1947,1.975925,2.414583,1900.125975,

In [2]:
# vol
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg', ascending=False)
stat.head(20)

,pattern,Pattern_use_vol,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Pattern_third_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
2,Pattern_Trend,1,1,5,0.6,0.55,0.5,28,0,171,2.494283,2.475417,255.522450,423.296250,1.7582,1.8732,2.2620,2.2135,2.2284,2.2297,2.2398,2.3086,2.3033,2.3453,2.3859,2.4150,2.4869,2.5218,2.6251,2.6805,2.7160,2.7816,2.7893,2.8024,2.7804,2.8284,2.9454,3.3421,1.57,1.41,1.31,1.50,1.65,1.75,1.77,2.04,2.19,2.27,2.30,2.40,2.49,2.71,2.83,2.91,3.02,3.08,3.21,3.29,3.32,3.38,3.48,3.53
0,Pattern_Trend,-1,1,5,0.6,0.55,0.5,28,0,1426,2.018821,2.420417,1452.838508,3451.514167,1.4254,1.4399,1.4987,1.5591,1.6241,1.6751,1.7664,1.8140,1.8395,1.8951,1.9038,1.9162,2.1611,2.1888,2.2157,2.2545,2.3154,2.3731,2.4062,2.4142,2.4267,2.4483,2.4363,2.4541,1.48,1.40,1.58,1.68,1.82,1.91,2.01,2.07,2.15,2.24,2.29,2.35,2.46,2.54,2.64,2.75,2.83,2.93,3.02,3.07,3.13,3.20,3.24,3.30
1,Pattern_Trend,0,1,5,0.6,0.55,0.5,28,0,1811,2.000754,2.346250,1812.365796,4249.058750,1.4406,1.4466,1.5358,1.5882,1.6322,1.6810,1.7722,1.8106,1.8295,1.8732,1.8816,1.9048,2.1077,2.1403,2.1843,2.2179,2.2753,2.3308,2.3457,2.3550,2.3797,2.4064,2.4112,2.4675,1.44,1.36,1.50,1.62,1.76,1.82,1.91,1.98,2.08,2.17,2.22,2.28,2.39,2.48,2.58,2.67,2.76,2.84,2.93,2.98,3.04,3.11,3.17,3.22


In [3]:
# high/low bound
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg', ascending=False)
stat.head(20)

,pattern,Pattern_use_vol,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Pattern_third_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
4,Pattern_Trend,-1,2,4,0.6,0.55,0.5,28,0,1170,2.002967,2.489583,1173.471000,2912.812500,1.4647,1.4565,1.4502,1.5239,1.6046,1.6413,1.7470,1.7909,1.8054,1.8767,1.8733,1.8841,2.1650,2.1826,2.2090,2.2469,2.3101,2.3597,2.3905,2.4008,2.4148,2.4332,2.4032,2.4368,1.54,1.49,1.63,1.72,1.86,1.92,2.03,2.10,2.21,2.32,2.38,2.45,2.55,2.62,2.71,2.83,2.91,3.02,3.10,3.13,3.20,3.29,3.34,3.40
0,Pattern_Trend,-1,1,4,0.6,0.55,0.5,28,0,1214,2.002767,2.465000,1217.358733,2992.510000,1.4592,1.4437,1.4699,1.5315,1.6086,1.6456,1.7481,1.7921,1.8079,1.8764,1.8809,1.8930,2.1731,2.1811,2.2062,2.2426,2.3033,2.3520,2.3858,2.3987,2.4074,2.4262,2.4009,2.4322,1.47,1.41,1.57,1.66,1.80,1.87,1.98,2.06,2.17,2.28,2.34,2.41,2.53,2.61,2.71,2.83,2.91,3.02,3.11,3.16,3.22,3.30,3.34,3.40
8,Pattern_Trend,-1,3,4,0.6,0.55,0.5,28,0,739,1.734888,2.447083,543.081863,1808.394583,1.4369,1.3624,1.3157,1.3642,1.4395,1.4846,1.5945,1.6454,1.6783,1.7308,1.6861,1.7010,1.7249,1.7470,1.7936,1.8611,1.9308,1.9473,1.9827,2.0317,2.0487,2.0577,2.0234,2.0490,1.59,1.32,1.50,1.60,1.75,1.86,2.01,2.07,2.18,2.28,2.26,2.38,2.47,2.55,2.67,2.79,2.89,3.03,3.09,3.17,3.22,3.29,3.34,3.42
5,Pattern_Trend,-1,2,5,0.6,0.55,0.5,28,0,1407,2.007988,2.432917,1418.238413,3423.113750,1.4210,1.4450,1.4732,1.5427,1.6150,1.6646,1.7604,1.8055,1.8294,1.8870,1.8925,1.9051,2.1463,2.1763,2.2051,2.2431,2.3051,2.3662,2.3940,2.3990,2.4156,2.4365,2.4213,2.4418,1.54,1.47,1.64,1.73,1.87,1.94,2.05,2.10,2.18,2.26,2.32,2.37,2.46,2.54,2.63,2.74,2.81,2.91,2.99,3.04,3.10,3.18,3.23,3.29
1,Pattern_Trend,-1,1,5,0.6,0.55,0.5,28,0,1453,2.008925,2.412500,1465.968025,3505.362500,1.4176,1.4348,1.4890,1.5483,1.6171,1.6677,1.7602,1.8070,1.8318,1.8873,1.8990,1.9126,2.1540,2.1756,2.2032,2.2428,2.3028,2.3626,2.3930,2.4002,2.4122,2.4333,2.4216,2.4405,1.48,1.41,1.58,1.68,1.81,1.90,2.00,2.06,2.14,2.23,2.28,2.34,2.45,2.53,2.62,2.74,2.82,2.92,3.01,3.06,3.12,3.19,3.24,3.29
9,Pattern_Trend,-1,3,5,0.6,0.55,0.5,28,0,979,1.804833,2.370833,787.931833,2321.045833,1.3836,1.3704,1.3787,1.4269,1.4901,1.5514,1.6469,1.7005,1.7420,1.7806,1.7546,1.7706,1.7982,1.8367,1.8827,1.9477,2.0151,2.0545,2.0855,2.1196,2.1385,2.1528,2.1386,2.1498,1.57,1.34,1.53,1.64,1.78,1.90,2.04,2.07,2.13,2.20,2.19,2.27,2.36,2.44,2.56,2.67,2.75,2.87,2.94,3.02,3.07,3.13,3.18,3.25
7,Pattern_Trend,-1,2,7,0.6,0.55,0.5,28,0,1835,1.931667,2.355000,1709.608333,4321.425000,1.3604,1.4027,1.4674,1.5280,1.6016,1.6606,1.7270,1.7631,1.7899,1.8325,1.8388,1.8572,2.0531,2.0756,2.1016,2.1344,2.1795,2.2395,2.2586,2.2623,2.2829,2.3060,2.3027,2.3346,1.52,1.45,1.60,1.70,1.84,1.94,2.00,2.06,2.14,2.19,2.24,2.30,2.39,2.45,2.54,2.64,2.72,2.80,2.87,2.91,2.97,3.03,3.09,3.13
3,Pattern_Trend,-1,1,7,0.6,0.55,0.5,28,0,1879,1.928375,2.341250,1744.416625,4399.208750,1.3607,1.3979,1.4811,1.5345,1.6042,1.6631,1.7251,1.7621,1.7894,1.8306,1.8355,1.8549,2.0523,2.0691,2.0919,2.1263,2.1703,2.2296,2.2511,2.2566,2.2739,2.2973,2.2965,2.3270,1.48,1.41,1.56,1.66,1.80,1.91,1.97,2.03,2.11,2.17,2.22,2.28,2.38,2.45,2.54,2.64,2.72,2.80,2.88,2.93,2.98,3.04,3.09,3.14
6,Pattern_Trend,-1,2,6,0.6,0.55,0.5,28,0,1635,1.941333,2.338333,1539.080000,3823.175000,1

# Research parameter influence

In [3]:
param = 'Pattern_use_vol'

res = stat.groupby([param]).agg({'e_ratio_avg': 'mean',
                                 'pct_price_diff_avg': 'mean',
                                 'e_ratio_rank': 'mean', 
                                 'price_rank': 'mean', 
                                 'forecasts_num': 'sum'}).sort_values(param).reset_index()
res

,Pattern_use_vol,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank,forecasts_num
0,-1,1.546973,2.873992,1061.440504,5577.185392,291081
1,0,1.387825,2.958303,865.087474,6598.829933,334592
2,1,1.387787,2.958333,865.153912,6600.041667,334650


# Plot result in file

In [23]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

x, y = param, 'e_ratio_avg'
ax.plot(res[x], res[y])
ax.set_xlabel(x)
ax.set_ylabel(y)
ax.set_title('_'.join(pattern) + '_' + ttype)

fig.savefig(f"{'_'.join(pattern)}'_'{ttype}_{param}", dpi=fig.dpi)

# Save new config data to config file

In [4]:
from config_updater import ConfigUpdater

ttype = 'buy'
pattern = ['Pattern', 'Trend']
work_timeframe = '15m'
higher_timeframe = '1h'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {
                 'Pattern': {
                            'use_vol': [-1], 
                            'window_low_bound': [1], 
                            'window_high_bound': [4],
                            'first_candle': [0.6],
                            'second_candle': [0.55],
                            'third_candle': [0.5]
                            },
                'Trend': {
                            'timeperiod': [28], 
                            'low_bound': [0]
                            }
             }
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)